2. Compare the model with DenseNet model
Load CIFAR-10 data set

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Load the CIFAR-10 dataset
(train_images_temp, train_labels_temp), (test_images_temp, test_labels_temp) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images_temp, test_images_temp = train_images_temp / 255.0, test_images_temp / 255.0

# Print dataset shapes
print("Train images shape:", train_images_temp.shape)
print("Train labels shape:", train_labels_temp.shape)
print("Test images shape:", test_images_temp.shape)
print("Test labels shape:", test_labels_temp.shape)

#merge data to a common data set

import numpy as np

images = np.concatenate((train_images_temp,test_images_temp),axis=0)
lables = np.concatenate((train_labels_temp,test_labels_temp),axis=0)

num_train_images = int(images.shape[0]*0.6)
num_validate_images = int(images.shape[0]*0.2)
num_test_images = int(images.shape[0]*0.2)

train_images,validate_images,test_images = np.split(images,[num_train_images,num_validate_images+num_test_images])
train_labels,validate_labels,test_labels = np.split(lables,[num_train_images,num_test_images+num_validate_images])



Train images shape: (50000, 32, 32, 3)
Train labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)


Import the DenseNet Model and train the model

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121  
from tensorflow.keras.layers import Input


#change the new input shape
new_input_shape = (32,32,3)

#new input layer
new_input_layer = Input(shape=new_input_shape)
# Load the pre-trained DenseNet model 
densenet_model = DenseNet121(weights='imagenet', include_top=True,input_tensor = new_input_layer)

# Optionally, set the model to evaluation mode
densenet_model.trainable = True

# View a summary of the model's architecture
densenet_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 38, 38, 3)            0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 16, 16, 64)           9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 16, 16, 64)           256       ['conv1/conv[0][0]']

In [3]:
import torch.nn as nn
densenet_model.fc = nn.Linear(in_features=64, out_features=10)

In [4]:
# Freeze all layers except the last one (custom classifier)
for layer in densenet_model.layers[:-1]:  
    layer.trainable = False

In [5]:
densenet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [6]:
densenet_model.fit(train_images, train_labels, validation_data=(validate_images, validate_labels), epochs=20, batch_size=32)

Epoch 1/20
1125/1125 [==============================] - 98s 82ms/step - loss: 1.7055 - accuracy: 0.4788
Epoch 2/20
1125/1125 [==============================] - 98s 87ms/step - loss: 1.1415 - accuracy: 0.6057
Epoch 3/20
1125/1125 [==============================] - 95s 84ms/step - loss: 1.0755 - accuracy: 0.6267
Epoch 4/20
1125/1125 [==============================] - 96s 85ms/step - loss: 1.0473 - accuracy: 0.6357
Epoch 5/20
1125/1125 [==============================] - 97s 86ms/step - loss: 1.0316 - accuracy: 0.6387
Epoch 6/20
1125/1125 [==============================] - 111s 99ms/step - loss: 1.0191 - accuracy: 0.6450
Epoch 7/20
1125/1125 [==============================] - 112s 99ms/step - loss: 1.0101 - accuracy: 0.6479
Epoch 8/20
1125/1125 [==============================] - 118s 105ms/step - loss: 1.0037 - accuracy: 0.6519
Epoch 9/20
1125/1125 [==============================] - 113s 100ms/step - loss: 0.9989 - accuracy: 0.6527
Epoch 10/20
1125/1125 [==============================] - 1